<a href="https://colab.research.google.com/github/bradneve/first_machine_learning-_model_final_project/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [465]:
import requests
import json
import csv
import pandas as pd
from numpy import nan
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
name_response = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/")
names = name_response.json()["elements"]

In [627]:
games_data = []
for i in range(1, len(names)):
  for name in names:
    if name["id"] == i:
      bootstrap_player_data = name
      break
  response = requests.get(f"https://fantasy.premierleague.com/api/element-summary/{i}/")
  player_games_data = response.json()
  prev_game = {"influence":0, "summed_influence":0, "creativity":0, "summed_creativity":0, "threat":0, "summed_threat":0,"summed_ict": 0, "ict_index": 0, "minutes": 0, "total_mins": 0, "total_goals": 0, "goals_scored": 0, "goals_conceded": 0, "total_goals_conceded": 0, "total_assists": 0, "assists": 0}
  for x in player_games_data["history"]:
    x["position"] = bootstrap_player_data["element_type"]
    x["id"] = bootstrap_player_data["id"]
    if x["minutes"] == 0 and prev_game["total_mins"] == 0:
          x["value"] = float(x["value"])
          x["opponent_team"] = float(x["opponent_team"])
          x["gameweek"] = float(x["round"])
          x["influence"] = float(x["influence"])
          x["creativity"] = float(x["creativity"])
          x["threat"] = float(x["threat"])
          x["summed_influence"] = prev_game["summed_influence"] + prev_game["influence"]
          x["summed_creativity"] = prev_game["summed_creativity"] + prev_game["creativity"]
          x["summed_threat"] = prev_game["summed_threat"] + prev_game["threat"]
          x["ict_index"] = float(x["ict_index"])
          x["minutes"] = float(x["minutes"])
          x["summed_ict"] = prev_game["summed_ict"] + prev_game["ict_index"]
          x["total_mins"] = prev_game["minutes"] + prev_game["total_mins"]
          x["total_goals"] = prev_game["goals_scored"] + prev_game["total_goals"]
          x["total_goals_conceded"] = prev_game["goals_conceded"] + prev_game["total_goals_conceded"]
          x["total_assists"] = prev_game["assists"] + prev_game["total_assists"]
          x["average_influence_per_90"] = round(prev_game["summed_influence"], 1)
          x["average_creativity_per_90"] = round(prev_game["summed_creativity"], 1)
          x["average_threat_per_90"] = round(prev_game["summed_threat"], 1)
          x["average_ict_per_90"] = round(prev_game["summed_ict"], 1)
          x["average_goals_per_90"] = round(prev_game["total_goals"], 1)
          x["average_goals_conceded_per_90"] = round(prev_game["total_goals_conceded"], 1)
          x["average_assists_per_90"] = round(prev_game["total_assists"], 1)
          x["average_mins"] = (prev_game["minutes"] + prev_game["total_mins"])/(player_games_data["history"].index(x)+1)
          games_data.append(x)
          prev_game = x
    elif prev_game["total_mins"] == 0 :
      x["value"] = float(x["value"])
      x["opponent_team"] = float(x["opponent_team"])
      x["gameweek"] = float(x["round"])
      x["influence"] = float(x["influence"])
      x["creativity"] = float(x["creativity"])
      x["threat"] = float(x["threat"])
      x["summed_influence"] = prev_game["summed_influence"] + prev_game["influence"]
      x["summed_creativity"] = prev_game["summed_creativity"] + prev_game["creativity"]
      x["summed_threat"] = prev_game["summed_threat"] + prev_game["threat"]
      x["ict_index"] = float(x["ict_index"])
      x["minutes"] = float(x["minutes"])
      x["summed_ict"] = prev_game["summed_ict"] + prev_game["ict_index"]
      x["total_mins"] = prev_game["minutes"] + prev_game["total_mins"]
      x["total_goals"] = prev_game["goals_scored"] + prev_game["total_goals"]
      x["total_goals_conceded"] = prev_game["goals_conceded"] + prev_game["total_goals_conceded"]
      x["total_assists"] = prev_game["assists"] + prev_game["total_assists"]
      x["average_influence_per_90"] = round(90*prev_game["influence"]/x["minutes"], 1)
      x["average_creativity_per_90"] = round(90*prev_game["creativity"]/x["minutes"], 1)
      x["average_threat_per_90"] = round(90*prev_game["threat"]/x["minutes"], 1)
      x["average_ict_per_90"] = round(90*prev_game["ict_index"]/x["minutes"], 1)
      x["average_goals_per_90"] = round(90*prev_game["goals_scored"]/x["minutes"], 1)
      x["average_goals_conceded_per_90"] = round(90*prev_game["goals_conceded"]/x["minutes"], 1)
      x["average_assists_per_90"] = round(90*prev_game["assists"]/x["minutes"], 1)
      x["average_mins"] = (prev_game["minutes"] + prev_game["total_mins"])/(player_games_data["history"].index(x)+1)
      games_data.append(x)
      prev_game = x
    else:
      x["value"] = float(x["value"])
      x["opponent_team"] = float(x["opponent_team"])
      x["influence"] = float(x["influence"])
      x["gameweek"] = float(x["round"])
      x["creativity"] = float(x["creativity"])
      x["threat"] = float(x["threat"])
      x["minutes"] = float(x["minutes"])
      x["summed_influence"] = prev_game["summed_influence"] + prev_game["influence"]
      x["summed_creativity"] = prev_game["summed_creativity"] + prev_game["creativity"]
      x["summed_threat"] = prev_game["summed_threat"] + prev_game["threat"]
      x["ict_index"] = float(x["ict_index"])
      x["summed_ict"] = prev_game["summed_ict"] + prev_game["ict_index"]
      x["total_mins"] = prev_game["minutes"] + prev_game["total_mins"]
      x["total_goals"] = prev_game["goals_scored"] + prev_game["total_goals"]
      x["total_assists"] = prev_game["assists"] + prev_game["total_assists"]
      x["total_goals_conceded"] = prev_game["goals_conceded"] + prev_game["total_goals_conceded"]
      x["average_influence_per_90"] = round(90*prev_game["summed_influence"]/prev_game["total_mins"], 1)
      x["average_creativity_per_90"] = round(90*prev_game["summed_creativity"]/prev_game["total_mins"], 1)
      x["average_threat_per_90"] = round(90*prev_game["summed_threat"]/prev_game["total_mins"], 1)
      x["average_ict_per_90"] = round(90*prev_game["summed_ict"]/prev_game["total_mins"], 1)
      x["average_goals_per_90"] = round(90*prev_game["total_goals"]/prev_game["total_mins"], 1)
      x["average_goals_conceded_per_90"] = round(90*prev_game["total_goals_conceded"]/prev_game["total_mins"], 1)
      x["average_assists_per_90"] = round(90*prev_game["total_assists"]/prev_game["total_mins"], 1)
      x["average_mins"] = (prev_game["minutes"] + prev_game["total_mins"])/(player_games_data["history"].index(x))
      games_data.append(x)
      prev_game = x
    



# print(games_data)



In [425]:
def select_data(player):
    return {
        "id": player["id"],
        "gameweek": player["gameweek"],
        "total_points": player["total_points"],
        "position": player["position"],
        "was_home": player["was_home"],
        "opponent_team": player["opponent_team"],
        "average_goals_per_90": player["average_goals_per_90"],
        "average_assists_per_90": player["average_assists_per_90"],
        "average_ict_per_90": player["average_ict_per_90"],
        "average_goals_conceded_per_90": player["average_goals_conceded_per_90"],
        "average_influence_per_90": player["average_influence_per_90"],
        "average_creativity_per_90": player["average_creativity_per_90"],
        "average_threat_per_90": player["average_threat_per_90"],
        "value": player["value"]
    }

for i in range(len(games_data)):
  games_data[i] = select_data(games_data[i])

# print(games_data)

In [628]:
headers = ['id', 'gameweek', 'total_points', "position", 'was_home', 'opponent_team', 'average_goals_per_90', "average_assists_per_90", "average_ict_per_90", "average_goals_conceded_per_90", "value", "average_influence_per_90", "average_creativity_per_90", "average_threat_per_90", "average_mins"]

with open("player_data.csv", "w", newline='') as file:
    f = csv.writer(file)
    f.writerow(headers)
    for p in games_data:
        values = [p[h] for h in headers]
        f.writerow(values)

In [629]:
player_data = pd.read_csv('player_data.csv')
player_data.sample(15)

,id,gameweek,total_points,position,was_home,opponent_team,average_goals_per_90,average_assists_per_90,average_ict_per_90,average_goals_conceded_per_90,value,average_influence_per_90,average_creativity_per_90,average_threat_per_90,average_mins
80,7,9.0,2,3,True,18.0,0.5,0.0,9.0,1.1,65.0,28.5,29.5,31.6,70.857143
6847,544,9.0,0,2,False,12.0,0.0,0.0,0.0,0.0,40.0,9.0,0.0,0.0,0.333333
2691,215,11.0,0,2,True,3.0,0.0,0.4,4.1,1.5,45.0,28.1,9.4,3.4,67.111111
5027,403,2.0,0,3,True,11.0,0.0,0.0,0.0,0.0,55.0,0.0,0.0,0.0,0.500000
2131,172,5.0,0,3,True,4.0,0.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.000000
3988,321,2.0,2,3,True,11.0,0.0,0.0,2.4,4.0,45.0,12.0,4.3,8.0,45.000000
5133,411,4.0,2,4,True,14.0,0.0,0.0,2.2,2.0,64.0,5.2,13.5,3.0,40.333333
6806,541,3.0,0,4,False,9.0,0.0,15.0,15.0,0.0,45.0,0.0,0.0,150.0,3.000000
6476,515,8.0,0,3,True,1.0,0.0,0.6,7.0,1.2,53.0,17.5,26.2,26.4,34.666667
4915,394,8.0,2,4,True,9.0,0.2,0.0,3.2,1.9,59.0,8.9,6.5,16.6,83.500000


In [630]:
player_data.columns

Index(['id', 'gameweek', 'total_points', 'position', 'was_home',
       'opponent_team', 'average_goals_per_90', 'average_assists_per_90',
       'average_ict_per_90', 'average_goals_conceded_per_90', 'value',
       'average_influence_per_90', 'average_creativity_per_90',
       'average_threat_per_90', 'average_mins'],
      dtype='object')

In [631]:
y = player_data["total_points"]

# X = player_data[["average_goals_per_90", "average_assists_per_90"]]
X = player_data[['position', 'opponent_team', 'was_home', 'average_goals_per_90',
       'average_assists_per_90', 'average_ict_per_90', 'average_goals_conceded_per_90', 'value', 'average_influence_per_90',
                'average_creativity_per_90', 'average_threat_per_90', 'average_mins']]


In [632]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=300)

In [633]:
model = DecisionTreeClassifier()

In [634]:
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [635]:
model.score(X_test, y_test)

0.5637755102040817

In [636]:
model.predict(X_test).sum()/len(model.predict(X_test))

1.25

In [ ]:
sns.lmplot(data=player_data[player_data["position"] == 2], x="total_points", y="average_goals_conceded_per_90", hue="position")

In [638]:
most_recent_gameweek_stats = [player for player in games_data if player["gameweek"] == 14 ]
print(most_recent_gameweek_stats)
data_to_predict_from = []


for player in most_recent_gameweek_stats:
  player_stats_to_predict_from = {}
  player_stats_to_predict_from["opponent_team"] = 3
  if player["total_mins"] == 0:
    player_stats_to_predict_from["id"] = float(player["id"])
    player_stats_to_predict_from["value"] = float(player["value"])
    player_stats_to_predict_from["position"] = float(player["position"])
    player_stats_to_predict_from["was_home"] = (player["was_home"])
    player_stats_to_predict_from["average_creativity_per_90"] = 0 
    player_stats_to_predict_from["average_influence_per_90"] = 0 
    player_stats_to_predict_from["average_threat_per_90"] = 0 
    player_stats_to_predict_from["average_ict_per_90"] = 0
    player_stats_to_predict_from["average_goals_per_90"] = 0 
    player_stats_to_predict_from["average_goals_conceded_per_90"] = 0
    player_stats_to_predict_from["average_assists_per_90"] = 0 
    player_stats_to_predict_from["average_mins"] = 0 
    data_to_predict_from.append(player_stats_to_predict_from)
  else:
    player_stats_to_predict_from["id"] = float(player["id"])
    player_stats_to_predict_from["value"] = float(player["value"])
    player_stats_to_predict_from["position"] = float(player["position"])
    player_stats_to_predict_from["was_home"] = (player["was_home"])
    player_stats_to_predict_from["average_creativity_per_90"] = 90*player["summed_creativity"]/player["total_mins"]
    player_stats_to_predict_from["average_influence_per_90"] = 90*player["summed_influence"]/player["total_mins"]
    player_stats_to_predict_from["average_threat_per_90"] = 90*player["summed_threat"]/player["total_mins"]
    player_stats_to_predict_from["average_ict_per_90"] = 90*player["summed_ict"]/player["total_mins"]
    player_stats_to_predict_from["average_goals_per_90"] = 90*player["total_goals"]/player["total_mins"]
    player_stats_to_predict_from["average_goals_conceded_per_90"] = 90*player["total_goals_conceded"]/player["total_mins"]
    player_stats_to_predict_from["average_assists_per_90"] = 90*player["total_assists"]/player["total_mins"]
    player_stats_to_predict_from["average_mins"] = player["average_mins"]
    data_to_predict_from.append(player_stats_to_predict_from)

print(data_to_predict_from)


[{'element': 1, 'fixture': 131, 'opponent_team': 16.0, 'total_points': 1, 'was_home': True, 'kickoff_time': '2022-10-30T14:00:00Z', 'team_h_score': 5, 'team_a_score': 0, 'round': 14, 'minutes': 27.0, 'goals_scored': 0, 'assists': 0, 'clean_sheets': 0, 'goals_conceded': 0, 'own_goals': 0, 'penalties_saved': 0, 'penalties_missed': 0, 'yellow_cards': 0, 'red_cards': 0, 'saves': 0, 'bonus': 0, 'bps': 2, 'influence': 2.4, 'creativity': 1.6, 'threat': 0.0, 'ict_index': 0.4, 'value': 42.0, 'transfers_balance': -247, 'selected': 14398, 'transfers_in': 137, 'transfers_out': 384, 'position': 2, 'id': 1, 'gameweek': 14.0, 'summed_influence': 0.0, 'summed_creativity': 0.0, 'summed_threat': 0.0, 'summed_ict': 0.0, 'total_mins': 0.0, 'total_goals': 0, 'total_goals_conceded': 0, 'total_assists': 0, 'average_influence_per_90': 0.0, 'average_creativity_per_90': 0.0, 'average_threat_per_90': 0.0, 'average_ict_per_90': 0.0, 'average_goals_per_90': 0.0, 'average_goals_conceded_per_90': 0.0, 'average_assis

In [639]:
headers = ["position", 'was_home', "opponent_team", 'average_goals_per_90', "average_assists_per_90", "average_ict_per_90", "average_goals_conceded_per_90", "value", "average_influence_per_90", "average_creativity_per_90", "average_threat_per_90", "average_mins"]

with open("data_to_predict_from_csv.csv", "w", newline='') as file:
    f = csv.writer(file)
    f.writerow(headers)
    for p in data_to_predict_from:
        values = [p[h] for h in headers]
        f.writerow(values)

In [640]:
data_to_predict_from_csv = pd.read_csv('data_to_predict_from_csv.csv')

In [662]:
predicted_scores = model.predict(data_to_predict_from_csv)
print(predicted_scores)

[ 0 10  5  0  0  3  2  1  0  6  1  0  2  0  2  1  0  0  7  0  1  0  0  0
  0  5  0  6  1  8  7  1  6  0  0  0  1  0  0  6  0  2  4  0  2 11  1  0
  0  0  0  0  0  2  0  0  0  0  1  0  0 13  0  2  0  2  1  9  2  5  6  2
  0  0  1  2  0  0  0  1  6  0  1  6  6  2  0  2  3  0  0  1  2  0  2  0
  0  0  0  0  2  0  2  1  0  1  3  2  0  1  3  2  5  1  0  2  0  0  1  2
  0  0  1  5  0  0  2  2  0  3  0  1  0  0  1  2  0  0  1  1  0  2  0  0
  6  0  2  5  0  1  0  2  0  0  1  6  0  1  0 13  1  0  1  3  6  6  2  0
  3 11  3  0  1  0  0  0  0  0  0  0  0  6  0  0  0 12  0  0  3  1  2  0
  1  6  7  0  0  0  6  1  0  0  0  3  1  0  0  0  0  6  3  0  0  0  0  0
  2  0  0  2  0  0  1  0  1  0  4  4  0  1  0  2  3  0  2  0  1 11  1  6
  0  1  0  2  1  3  0  0  2  0  0  3  0  3  2  0  2  2  1  1  2  1  0  8
  0  2  0  6  0  9  1  0  0 11  1  0  1  0  2  1  2  2  8  6  8  2  0  0
  0  0  1  0  0  0  2  1 -1  0  4  3 11  7  1  1  1  1  3  0  1  0  2  1
  0  1  2  1  0 13  1  0  0  0  0  0  0  1 15  0  2

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [663]:
result = np.where( predicted_scores == 15)
print(result)
# print(data_to_predict_from[1])
print(data_to_predict_from[317])

(array([326]),)
{'opponent_team': 3, 'id': 318.0, 'value': 122.0, 'position': 4.0, 'was_home': False, 'average_creativity_per_90': 13.930434782608696, 'average_influence_per_90': 62.68695652173914, 'average_threat_per_90': 82.17391304347827, 'average_ict_per_90': 15.88695652173913, 'average_goals_per_90': 1.6630434782608696, 'average_goals_conceded_per_90': 1.076086956521739, 'average_assists_per_90': 0.29347826086956524, 'average_mins': 83.63636363636364}


# Trying to predict One player

In [553]:
haaland_data = []
for x in names:
  if x["id"] == 427:
    boostrap_player_data = x
    break
response = requests.get(f"https://fantasy.premierleague.com/api/element-summary/427/")
player_haaland_data = response.json()
prev_game = {"influence":0, "summed_influence":0, "creativity":0, "summed_creativity":0, "threat":0, "summed_threat":0,"summed_ict": 0, "ict_index": 0, "minutes": 0, "total_mins": 0, "total_goals": 0, "goals_scored": 0, "goals_conceded": 0, "total_goals_conceded": 0, "total_assists": 0, "assists": 0}
for x in player_haaland_data["history"]:
  x["position"] = bootstrap_player_data["element_type"]
  x["id"] = bootstrap_player_data["id"]
  if x["minutes"] == 0 and prev_game["total_mins"] == 0:
        x["value"] = float(x["value"])
        x["gameweek"] = float(x["round"])
        x["influence"] = float(x["influence"])
        x["creativity"] = float(x["creativity"])
        x["threat"] = float(x["threat"])
        x["summed_influence"] = prev_game["summed_influence"] + prev_game["influence"]
        x["summed_creativity"] = prev_game["summed_creativity"] + prev_game["creativity"]
        x["summed_threat"] = prev_game["summed_threat"] + prev_game["threat"]
        x["ict_index"] = float(x["ict_index"])
        x["minutes"] = float(x["minutes"])
        x["summed_ict"] = prev_game["summed_ict"] + prev_game["ict_index"]
        x["total_mins"] = prev_game["minutes"] + prev_game["total_mins"]
        x["total_goals"] = prev_game["goals_scored"] + prev_game["total_goals"]
        x["total_goals_conceded"] = prev_game["goals_conceded"] + prev_game["total_goals_conceded"]
        x["total_assists"] = prev_game["assists"] + prev_game["total_assists"]
        x["average_influence_per_90"] = round(prev_game["summed_influence"], -1)
        x["average_creativity_per_90"] = round(prev_game["summed_creativity"], -1)
        x["average_threat_per_90"] = round(prev_game["summed_threat"], -1)
        x["average_ict_per_90"] = round(prev_game["summed_ict"], -1)
        x["average_goals_per_90"] = round(prev_game["total_goals"], -1)
        x["average_goals_conceded_per_90"] = round(prev_game["total_goals_conceded"], -1)
        x["average_assists_per_90"] = round(prev_game["total_assists"], -1)
        haaland_data.append(x)
        prev_game = x
  elif prev_game["total_mins"] == 0 :
    x["value"] = float(x["value"])
    x["gameweek"] = float(x["round"])
    x["influence"] = float(x["influence"])
    x["creativity"] = float(x["creativity"])
    x["threat"] = float(x["threat"])
    x["summed_influence"] = prev_game["summed_influence"] + prev_game["influence"]
    x["summed_creativity"] = prev_game["summed_creativity"] + prev_game["creativity"]
    x["summed_threat"] = prev_game["summed_threat"] + prev_game["threat"]
    x["ict_index"] = float(x["ict_index"])
    x["minutes"] = float(x["minutes"])
    x["summed_ict"] = prev_game["summed_ict"] + prev_game["ict_index"]
    x["total_mins"] = prev_game["minutes"] + prev_game["total_mins"]
    x["total_goals"] = prev_game["goals_scored"] + prev_game["total_goals"]
    x["total_goals_conceded"] = prev_game["goals_conceded"] + prev_game["total_goals_conceded"]
    x["total_assists"] = prev_game["assists"] + prev_game["total_assists"]
    x["average_influence_per_90"] = round(90*prev_game["influence"]/x["minutes"], -1)
    x["average_creativity_per_90"] = round(90*prev_game["creativity"]/x["minutes"], -1)
    x["average_threat_per_90"] = round(90*prev_game["threat"]/x["minutes"], -1)
    x["average_ict_per_90"] = round(90*prev_game["ict_index"]/x["minutes"], -1)
    x["average_goals_per_90"] = round(90*prev_game["goals_scored"]/x["minutes"], -1)
    x["average_goals_conceded_per_90"] = round(90*prev_game["goals_conceded"]/x["minutes"], -1)
    x["average_assists_per_90"] = round(90*prev_game["assists"]/x["minutes"], -1)
    haaland_data.append(x)
    prev_game = x
  else:
    x["value"] = float(x["value"])
    x["influence"] = float(x["influence"])
    x["gameweek"] = float(x["round"])
    x["creativity"] = float(x["creativity"])
    x["threat"] = float(x["threat"])
    x["minutes"] = float(x["minutes"])
    x["summed_influence"] = prev_game["summed_influence"] + prev_game["influence"]
    x["summed_creativity"] = prev_game["summed_creativity"] + prev_game["creativity"]
    x["summed_threat"] = prev_game["summed_threat"] + prev_game["threat"]
    x["ict_index"] = float(x["ict_index"])
    x["summed_ict"] = prev_game["summed_ict"] + prev_game["ict_index"]
    x["total_mins"] = prev_game["minutes"] + prev_game["total_mins"]
    x["total_goals"] = prev_game["goals_scored"] + prev_game["total_goals"]
    x["total_assists"] = prev_game["assists"] + prev_game["total_assists"]
    x["total_goals_conceded"] = prev_game["goals_conceded"] + prev_game["total_goals_conceded"]
    x["average_influence_per_90"] = round(90*prev_game["summed_influence"]/prev_game["total_mins"], -1)
    x["average_creativity_per_90"] = round(90*prev_game["summed_creativity"]/prev_game["total_mins"], -1)
    x["average_threat_per_90"] = round(90*prev_game["summed_threat"]/prev_game["total_mins"], -1)
    x["average_ict_per_90"] = round(90*prev_game["summed_ict"]/prev_game["total_mins"], -1)
    x["average_goals_per_90"] = round(90*prev_game["total_goals"]/prev_game["total_mins"], -1)
    x["average_goals_conceded_per_90"] = round(90*prev_game["total_goals_conceded"]/prev_game["total_mins"], -1)
    x["average_assists_per_90"] = round(90*prev_game["total_assists"]/prev_game["total_mins"], -1)
    haaland_data.append(x)
    prev_game = x

In [554]:
print(haaland_data)

[{'element': 427, 'fixture': 8, 'opponent_team': 17, 'total_points': 2, 'was_home': True, 'kickoff_time': '2022-08-06T14:00:00Z', 'team_h_score': 4, 'team_a_score': 1, 'round': 1, 'minutes': 90.0, 'goals_scored': 0, 'assists': 0, 'clean_sheets': 0, 'goals_conceded': 1, 'own_goals': 0, 'penalties_saved': 0, 'penalties_missed': 0, 'yellow_cards': 0, 'red_cards': 0, 'saves': 0, 'bonus': 0, 'bps': 4, 'influence': 14.6, 'creativity': 22.2, 'threat': 34.0, 'ict_index': 7.1, 'value': 115.0, 'transfers_balance': 0, 'selected': 2286082, 'transfers_in': 0, 'transfers_out': 0, 'position': 1, 'id': 655, 'gameweek': 1.0, 'summed_influence': 0, 'summed_creativity': 0, 'summed_threat': 0, 'summed_ict': 0, 'total_mins': 0, 'total_goals': 0, 'total_goals_conceded': 0, 'total_assists': 0, 'average_influence_per_90': 0.0, 'average_creativity_per_90': 0.0, 'average_threat_per_90': 0.0, 'average_ict_per_90': 0.0, 'average_goals_per_90': 0.0, 'average_goals_conceded_per_90': 0.0, 'average_assists_per_90': 0

In [555]:
for i in range(len(haaland_data)):
  haaland_data[i] = select_data(haaland_data[i])

In [556]:
headers = ['total_points', "position", 'was_home', 'opponent_team', 'average_goals_per_90', "average_assists_per_90", "average_ict_per_90", "average_goals_conceded_per_90", "value", "average_influence_per_90", "average_creativity_per_90", "average_threat_per_90"]

with open("haaland_data_csv.csv", "w", newline='') as file:
    f = csv.writer(file)
    f.writerow(headers)
    for p in haaland_data:
        values = [p[h] for h in headers]
        f.writerow(values)

In [557]:
headers = ['total_points', "position", 'was_home', 'opponent_team', 'average_goals_per_90', "average_assists_per_90", "average_ict_per_90", "average_goals_conceded_per_90", "value", "average_influence_per_90", "average_creativity_per_90", "average_threat_per_90"]

with open("jesus_data_csv.csv", "w", newline='') as file:
    f = csv.writer(file)
    f.writerow(headers)
    for p in games_data:
        values = [p[h] for h in headers]
        f.writerow(values)

In [558]:
haaland_data_csv = pd.read_csv('haaland_data_csv.csv')
haaland_data_csv

,total_points,position,was_home,opponent_team,average_goals_per_90,average_assists_per_90,average_ict_per_90,average_goals_conceded_per_90,value,average_influence_per_90,average_creativity_per_90,average_threat_per_90
0,2,1,True,17,0.0,0.0,0.0,0.0,115.0,0.0,0.0,0.0
1,8,1,False,6,0.0,0.0,10.0,0.0,114.0,10.0,20.0,30.0
2,6,1,True,20,0.0,0.0,10.0,0.0,114.0,10.0,20.0,30.0
3,10,1,False,16,0.0,0.0,10.0,0.0,114.0,30.0,30.0,40.0
4,5,1,False,19,0.0,0.0,10.0,0.0,114.0,30.0,30.0,50.0
5,9,1,True,9,0.0,0.0,10.0,0.0,114.0,50.0,30.0,60.0
6,10,1,True,10,0.0,0.0,10.0,0.0,114.0,40.0,30.0,60.0
7,6,1,False,1,0.0,0.0,10.0,0.0,114.0,40.0,30.0,60.0
8,7,1,False,5,0.0,0.0,10.0,0.0,114.0,40.0,30.0,60.0
9,9,1,True,8,0.0,0.0,10.0,0.0,114.0,40.0,20.0,60.0


Average ict skewing results for low minute players